# Teste de engenharia de dados Banco ABC
Este notebook apresenta um exemplo de como transformar seus dados armazenados "in-memory" em tabelas no SQLite, permitindo a manipulação através de querys SQL. Caso ainda tenha dúvidas, você pode utilizar tutoriais [como esse](https://deepnote.com/blog/query-pandas-dataframes-with-sql) para ver mais detalhes. Esse snippet foi testado utilizando o Python 3.10.

O primeiro passo é importar as bibliotecas necessárias. Caso você não as tenha instalada na máquina, será necessário realizar isso antes.

In [106]:
import pandas as pd
import sqlalchemy
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Feito isso, você deve fazer a leitura das bases de dados para a memória. O exemplo abaixo é um pseudocódigo.

In [107]:
clients = pd.read_csv('data\clientes.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\User\AppData\Local\Temp\ipykernel_5516\2457124910.py:1: SyntaxWarning: invalid escape sequence '\c'
  clients = pd.read_csv('data\clientes.csv')


Depois, você irá inicializar a SQLite através do código abaixo:

In [108]:
engine = sqlalchemy.create_engine('sqlite:///mydatabase.db')
%sql sqlite:///mydatabase.db

Agora, basta transformar seus dados em tabelas efetivamente. É possível que você se depare com o seguinte erro: **ValueError: Unsigned 64 bit integer datatype is not supported**. Isso ocorre por que o SQLite não oferece suporte para o tipo **uint64**. Nesse caso, você deve tentar converter as colunas do seu dataset que estão em uint64 para string. As colunas que recebem esse tipo geralmente são *numeroCartao* e *numeroConta*.

In [109]:
clients.to_sql('clientes', con=engine, if_exists='replace')

250000

Finalmente, você deve ser capaz de executar querys utilizando a seguinte syntax:

In [110]:
query = %sql """ SELECT * \
                 FROM clientes limit 10"""

 * sqlite:///mydatabase.db
Done.


In [22]:
print(query)

+-------+-------------+------------------------+----------------+-----------------------------+------------------+--------------+
| index |     cpf     |          nome          | dataNascimento |            email            |       city       | dataCadastro |
+-------+-------------+------------------------+----------------+-----------------------------+------------------+--------------+
|   0   | 18046079841 |     Beatriz Porto      |   1988-01-13   |     ocorreia@cardoso.org    |     Ferreira     |  2012-06-05  |
|   1   | 81465703213 |  Luiz Miguel Carvalho  |   1996-10-02   |    theoviana@hotmail.com    |   Dias Grande    |  2011-11-27  |
|   2   | 58695342240 |      Calebe Sales      |   2003-09-28   |   luiz-otavioda-cruz@da.br  |   Costa Grande   |  2020-01-23  |
|   3   | 78857172116 |  Srta. Bárbara Duarte  |   1968-11-19   | silveiraisadora@martins.com | Ferreira de Melo |  2021-07-09  |
|   4   | 80101594884 |  Francisco Nascimento  |   1988-08-08   |      bda-luz@gmail.com  

## Primeira fase - Query 1
Extraia o nome, CPF e e-mail de todos os clientes que tenham ao menos R$ 400,00 reais de compras aprovadas nos últimos dois meses. Além disso, esses clientes precisam estar com a conta ativa e com o cartão desbloqueado, a menos que o código de bloqueio do cartão seja igual a “M”. 

In [111]:
# Tentar diferentes codificações
try:
    transactions = pd.read_csv('data/transacoes.csv', encoding='utf-8')
except UnicodeDecodeError:
    transactions = pd.read_csv('data/transacoes.csv', encoding='latin1')

In [112]:
# Carregar os DataFrames com o formato correto
accounts = pd.read_csv('data/contas.csv', delimiter=';')
cards = pd.read_excel('data/cartoes.xlsx')  # Ler o arquivo Excel
#transactions = pd.read_csv('data/transacoes.csv', delimiter=';')

In [113]:
# Converter colunas uint64 para int64
accounts['numeroConta'] = accounts['numeroConta'].astype('int64')
cards['numeroCartao'] = cards['numeroCartao'].astype('int64')
transactions['numeroCartao'] = transactions['numeroCartao'].astype('int64')

In [114]:
# Verificar os tipos de dados das colunas
print("Tipos de dados em accounts:", accounts.dtypes)
print("Tipos de dados em cards:", cards.dtypes)
print("Tipos de dados em transactions:", transactions.dtypes)


Tipos de dados em accounts: codMatricula    object
cpfCliente       int64
numeroConta      int64
agencia          int64
dataAbertura    object
ativo            int64
saldo            int64
dtype: object
Tipos de dados em cards: codMatricula                 object
numeroCartao                  int64
cartaoBloqueado                bool
codBloqueio                  object
nomeImprCart                 object
digitoVerificador             int64
dataEmisCartao       datetime64[ns]
limiteTotal                 float64
limiteRestante              float64
dtype: object
Tipos de dados em transactions: numeroCartao      int64
dataCompra       object
aprovado          int64
idLoja            int64
qtdParcelas       int64
valorCompra     float64
nomeLoja         object
dtype: object


In [115]:
# Inserir DataFrames no banco de dados
accounts.to_sql('contas', con=engine, if_exists='replace', index=False)
cards.to_sql('cartoes', con=engine, if_exists='replace', index=False)
transactions.to_sql('transacoes', con=engine, if_exists='replace', index=False)

1250000

In [116]:
# Obter a data mais recente de dataCompra
query_data_recente = %sql SELECT MAX(dataCompra) AS dataRecente FROM transacoes;

# Obter a data subtraída de dois meses
query_data_subtrair_dois_meses = %sql SELECT DATE(MAX(dataCompra), '-2 months') AS dataMenosDoisMeses FROM transacoes;

 * sqlite:///mydatabase.db
Done.
 * sqlite:///mydatabase.db
Done.


In [117]:
print(f'A data de compra mais recente da base é: {query_data_recente}')
print('')
print(f'A data de compra de dois meses para trás é: {query_data_subtrair_dois_meses}')

A data de compra mais recente da base é: +-------------+
| dataRecente |
+-------------+
|  2024-02-21 |
+-------------+

A data de compra de dois meses para trás é: +--------------------+
| dataMenosDoisMeses |
+--------------------+
|     2023-12-21     |
+--------------------+


In [124]:
query_1 = %sql SELECT clientes.nome, clientes.cpf, clientes.email FROM clientes JOIN contas ON clientes.cpf = contas.cpfCliente JOIN cartoes ON contas.codMatricula = cartoes.codMatricula JOIN transacoes ON cartoes.numeroCartao = transacoes.numeroCartao WHERE contas.ativo = 1 AND (cartoes.cartaoBloqueado = 0 OR cartoes.codBloqueio = 'M') AND transacoes.aprovado = 1 AND transacoes.dataCompra BETWEEN '2024-02-21' AND '2024-12-21' GROUP BY clientes.nome, clientes.cpf, clientes.email HAVING SUM(transacoes.valorCompra) >= 400;
print(query_1)

 * sqlite:///mydatabase.db
Done.
+------------------------------+-------------+-----------------------------------------+
|             nome             |     cpf     |                  email                  |
+------------------------------+-------------+-----------------------------------------+
|         Agatha Gomes         | 47030856073 |          larissa95@yahoo.com.br         |
|      Agatha Nascimento       | 53686156562 |         fogacaraquel@silveira.br        |
|         Agatha Nunes         | 36537807266 |             gmelo@freitas.br            |
|       Agatha Oliveira        | 65631925523 |            yasmin76@barros.br           |
|        Agatha Ribeiro        | 71612587231 |          matheus75@monteiro.org         |
|         Alana Farias         | 49928102324 |           wmartins@aragao.com           |
|        Alana Martins         | 92950456450 |           ybarros@hotmail.com           |
|        Alana Monteiro        | 45154499126 |           catarina06@ig.com.br

In [123]:
#Testes de consulta realizados
teste = %sql SELECT clientes.nome, clientes.cpf, clientes.email FROM clientes LIMIT 10
print(teste)

 * sqlite:///mydatabase.db
Done.
+------------------------+-------------+-----------------------------+
|          nome          |     cpf     |            email            |
+------------------------+-------------+-----------------------------+
|     Beatriz Porto      | 18046079841 |     ocorreia@cardoso.org    |
|  Luiz Miguel Carvalho  | 81465703213 |    theoviana@hotmail.com    |
|      Calebe Sales      | 58695342240 |   luiz-otavioda-cruz@da.br  |
|  Srta. Bárbara Duarte  | 78857172116 | silveiraisadora@martins.com |
|  Francisco Nascimento  | 80101594884 |      bda-luz@gmail.com      |
|      Laura da Luz      | 12215420147 |      yasmin37@gmail.com     |
|  Ana Carolina Freitas  | 35010549464 |  danielda-mota@yahoo.com.br |
| Maria Fernanda Pereira | 70452017421 |    lunafogaca@hotmail.com   |
|     Bryan da Rocha     | 31815402513 |      ssales@uol.com.br      |
|     Renan Silveira     | 14897402109 |       nicole06@da.org       |
+------------------------+-------------+----